In [13]:
# pip install -r requirements.txt

version of pandas


AttributeError: module 'socket' has no attribute '__version__'

In [14]:
import dpkt
import socket
import pandas as pd

# Initialize a list to store parsed packet data
packet_data = []

# Define the pcap file path
pcap_filepath = "../data/maccdc2012_00000.pcap"

# Open and read the pcap file
with open(pcap_filepath, 'rb') as f:
    pcap_reader = dpkt.pcap.Reader(f)
    
    for i, (ts, buf) in enumerate(pcap_reader):
        # Parse Ethernet frame
        eth = dpkt.ethernet.Ethernet(buf)
        
        # Check if packet contains IP data
        if isinstance(eth.data, dpkt.ip.IP):
            ip = eth.data
            
            # Check if IP data contains TCP/UDP data
            if isinstance(ip.data, dpkt.tcp.TCP) or isinstance(ip.data, dpkt.udp.UDP):
                transport_layer = ip.data
                protocol = 'TCP' if isinstance(transport_layer, dpkt.tcp.TCP) else 'UDP'

                # Extract fields and convert IPs to readable format
                packet_info = {
                    'timestamp': ts,
                    'src_ip': socket.inet_ntoa(ip.src),
                    'dst_ip': socket.inet_ntoa(ip.dst),
                    'src_port': transport_layer.sport,
                    'dst_port': transport_layer.dport,
                    'protocol': protocol,
                    'length': ip.len
                }
                
                # Append to the packet_data list
                packet_data.append(packet_info)
        
        # Break after a certain number of packets if needed
        if i >= 10:  # Adjust as needed for more or fewer packets
            break

# Convert packet data to a DataFrame for easier analysis
df = pd.DataFrame(packet_data)

In [16]:
print(df.head())
print("--------------------")
print(df.describe())
print("--------------------")
print(df.info())

      timestamp           src_ip           dst_ip  src_port  dst_port  \
0  1.331901e+09  192.168.229.254   192.168.202.79       443     46117   
1  1.331901e+09   192.168.202.79  192.168.229.254     46117       443   
2  1.331901e+09   192.168.202.79  192.168.229.251     50463        80   
3  1.331901e+09  192.168.229.254   192.168.202.79       443     46117   
4  1.331901e+09   192.168.202.79  192.168.229.251     50465        80   

  protocol  length  
0      TCP      99  
1      TCP     251  
2      TCP      52  
3      TCP      52  
4      TCP      60  
--------------------
          timestamp      src_port      dst_port      length
count  1.100000e+01     11.000000     11.000000   11.000000
mean   1.331901e+09  18599.363636  30232.818182  109.818182
std    0.000000e+00  25416.063134  23818.626882   67.263390
min    1.331901e+09     80.000000     80.000000   52.000000
25%    1.331901e+09    443.000000    444.000000   56.000000
50%    1.331901e+09    443.000000  46117.000000   99.0